In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups
from skcosmo.decomposition import PCovR

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Utilities
import h5py
import json
import itertools
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json, save_json
from tempfile import mkdtemp
from shutil import rmtree

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Load train and test splits

In [3]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [4]:
# Load train sets for IZA and Deem
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_230/svm_train.idxs', dtype=int)
iza_sort_idxs = np.argsort(iza_train_idxs)
iza_unsort_idxs = np.argsort(iza_sort_idxs)
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/svm_train.idxs', dtype=int)

In [5]:
# Load cantons for IZA and Deem
iza_cantons = np.loadtxt('../Raw_Data/IZA_230/cantons_compositions.dat', usecols=1, dtype=int)
deem_cantons_2 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_2-class.dat', dtype=int)
deem_cantons_4 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_4-class.dat', dtype=int)

In [6]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs], 
    deem_cantons_4[deem_train_idxs]
))

cantons[2] = np.concatenate((
    np.ones(len(iza_train_idxs), dtype=int),
    deem_cantons_2[deem_train_idxs]
))

# Build set of class weights (by sample) for centering and scaling
class_weights = {n_cantons: utils.balanced_class_weights(cantons[n_cantons]) for n_cantons in (2, 4)}

# Model setup

In [7]:
model_dir = '../Processed_Data/Models'

In [8]:
deem_name = 'DEEM_330k'
iza_name = 'IZA_230'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

# Linear PCovR

In [9]:
batch_size = 20000

In [10]:
# Linear model setup
n_species = 2
group_names = {
    'power': ['OO', 'OSi', 'SiSi', 'OO+OSi', 'OO+SiSi', 'OSi+SiSi', 'OO+OSi+SiSi'], 
    'radial': ['O', 'Si', 'O+Si']
}
# df_types = ['OvR', 'OvO']
df_types = ['OvO']
mixing_suffixes = ['', '_0.0', '_1.0']

In [11]:
for df_type in tqdm(df_types, desc='DF', leave=True):
    for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
        linear_dir = f'{model_dir}/{cutoff}/LPCovR/{df_type}'

        for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
            spectrum_name = spectrum_type.capitalize()

            # Load SOAPs
            iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
            iza_soaps = utils.load_hdf5(iza_file)

            deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
            f = h5py.File(deem_file, 'r')
            deem_330k_dataset = f['0']

            soaps = np.vstack((iza_soaps[iza_train_idxs], deem_330k_dataset[deem_train_idxs]))

            n_features = soaps.shape[1]
            feature_groups = extract_species_pair_groups(
                n_features, n_species, 
                spectrum_type=spectrum_type,
                combinations=True
            )

            # Prepare batches for PCovR
            n_samples_330k = deem_330k_dataset.len()
            n_batches = n_samples_330k // batch_size
            if n_samples_330k % batch_size > 0:
                n_batches += 1

            for species_pairing, feature_idxs in zip(
                tqdm(group_names[spectrum_type], desc='Species', leave=False),
                feature_groups
            ):

                for n_cantons in tqdm((2, 4), desc='Classes', leave=False):

                    # Prepare inputs and outputs
                    df_dir = f'LSVC/{df_type}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'

                    # Load decision functions
                    iza_dfs = np.loadtxt(f'{iza_dir}/{cutoff}/{df_dir}/svc_structure_dfs.dat')
                    iza_dfs = iza_dfs[iza_train_idxs]

                    deem_dfs = np.loadtxt(f'{deem_dir}/{cutoff}/{df_dir}/svc_structure_dfs.dat')
                    deem_dfs = deem_dfs[deem_train_idxs]

                    dfs = np.concatenate((iza_dfs, deem_dfs))

                    # Prepare outputs
                    output_dir = f'LPCovR/{df_type}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'

                    os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}', exist_ok=True)
                    os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}', exist_ok=True)

                    parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'

                    # Load estimator parameters
                    y_scaler_parameters = load_json(f'{parameter_dir}/y_scaler_parameters.json')
                    ridge_parameters = load_json(f'{parameter_dir}/ridge_parameters.json')
    #                 svc_parameters = load_json(f'{parameter_dir}/svc_parameters_balanced_accuracy.json')

                    for mixing in tqdm(mixing_suffixes, desc='Mixing', leave=False):
                        pcovr_parameters = load_json(f'{parameter_dir}/pcovr_parameters_score{mixing}.json')

                        # Move the optimized alpha (regularization) 
                        # from the PCovR parameter dictionary to those for Ridge
                        ridge_parameters['alpha'] = pcovr_parameters.pop('alpha')

                        # Model pipeline
                        pipeline = Pipeline(
                            [
                                ('norm_scaler', utils.StandardNormScaler()),
                                ('pcovr', utils.TransformedTargetRegressorTransformer(
                                    regressor=PCovR(
                                        **pcovr_parameters,
                                        regressor=Ridge(**ridge_parameters)
                                    ),
                                    transformer=utils.StandardNormScaler(**y_scaler_parameters)
                                ))
                            ],
                        )
                        repeat_idxs = utils.n_repeats(cantons[n_cantons])
                        pipeline.fit(
                            np.repeat(soaps[:, feature_idxs], repeat_idxs, axis=0),
                            np.repeat(dfs, repeat_idxs, axis=0)
                        )

                        # Read the IZA structures and compute decision functions
                        # and canton predictions
                        iza_projections = pipeline.transform(iza_soaps[:, feature_idxs])
                        iza_predicted_dfs = pipeline.predict(iza_soaps[:, feature_idxs])

                        # Can't pull df_type directly from the SVC parameters
                        # b/c we use the OneVsRestClassifier wrapper class
                        iza_predicted_cantons = utils.df_to_class(
                            iza_predicted_dfs, df_type=df_type.lower(), n_classes=n_cantons
                        )

                        utils.save_hdf5(
                            f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_projections{mixing}.hdf5', 
                            iza_projections, attrs=pcovr_parameters
                        )
                        np.savetxt(f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs{mixing}.dat', iza_predicted_dfs)
                        np.savetxt(f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons{mixing}.dat', iza_predicted_cantons, fmt='%d')

                        # Read the DEEM_330k structures and compute decision functions
                        # and canton predictions in batches
                        deem_projections = np.zeros((n_samples_330k, pcovr_parameters['n_components']))

                        if n_cantons == 2:
                            deem_predicted_dfs = np.zeros(n_samples_330k)
                        else:
                            if df_type == 'OvR':
                                deem_predicted_dfs = np.zeros((n_samples_330k, n_cantons))
                            elif df_type == 'OvO':
                                deem_predicted_dfs = np.zeros((n_samples_330k, n_cantons * (n_cantons - 1) // 2))

                        for i in tqdm(range(0, n_batches), desc='Batch', leave=False):
                            batch_slice = slice(i * batch_size, (i + 1) * batch_size)

                            deem_330k_batch = deem_330k_dataset[batch_slice, feature_idxs]                    
                            deem_projections[batch_slice] = pipeline.transform(deem_330k_batch)
                            deem_predicted_dfs[batch_slice] = pipeline.predict(deem_330k_batch)

                        # Squeeze the decision functions
                        deem_predicted_cantons = utils.df_to_class(
                            deem_predicted_dfs, df_type=df_type.lower(), n_classes=n_cantons
                        )

                        # Save DEEM 330k PCovR projections, decision functions, and canton predictions
                        utils.save_hdf5(
                            f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_projections{mixing}.hdf5', 
                            deem_projections, attrs=pcovr_parameters
                        )
                        np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs{mixing}.dat', deem_predicted_dfs)
                        np.savetxt(f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons{mixing}.dat', deem_predicted_cantons, fmt='%d')

                        # Save the model and the scaler
                        save_json(pipeline.named_steps['norm_scaler'].__dict__, f'{parameter_dir}/norm_scaler{mixing}.json', array_convert=True)
                        save_json(pipeline.named_steps['pcovr'].regressor_.__dict__, f'{parameter_dir}/pcovr_regressor{mixing}.json', array_convert=True)
                        save_json(pipeline.named_steps['pcovr'].transformer_.__dict__, f'{parameter_dir}/pcovr_transformer{mixing}.json', array_convert=True)